In [2]:
import $file.common
import common._, dc._
import doobie.implicits._

Compiling /home/juanshac/Documents/tagless-final-tutorial/common.sc

import $file.$     

import common._, dc._

import doobie.implicits._

# Variation 5. Quoted DSLs: Quill

This is an all completely different approach to our problem, based on so-called Quoted DSLs. Unlike MTL, Quoted DSLs don't build upon custom syntax declared through APIs but upon the host language AST (Abstract Syntax Tree, i.e. the Scala syntax itself). And semantics is not given through API instances but through AST transformations, i.e. macros. The application of Quoted DSLs to Language-Integrated Query was one of the key features of the following [paper](http://homepages.inf.ed.ac.uk/slindley/papers/practical-theory-of-linq.pdf): 

![](images/practicalLinq.png)

This paper explains how to solve the _avalanche_ query problem through a set of rewriting rules over the original query. The Scala library `quill` put this idea into practice. From the user point of view, a data model in `quill` is programmed through a flat model of case classes: 

In [3]:
case class Country(
    code: String, 
    name: String, 
    capital: Option[Int])

case class City(
    id: Int, 
    name: String, 
    countryCode: String, 
    population: Int)

defined class Country
defined class City

Then, queries are written using common Scala for-comprehensions. In our case:

In [4]:
val largeCapitals = quote { 
    for {
        country <- query[Country] 
        city <- query[City] 
        if country.capital.exists(_ == city.id) 
        if city.population > 8000000
    } yield (country.name, city.name) 
}

largeCapitals: AnyRef with Quoted[Query[(String, String)]]{def quoted: io.getquill.ast.FlatMap;def ast: io.getquill.ast.FlatMap;def id586702505(): Unit;val liftings: Object} = querySchema("Country").flatMap(country => querySchema("City").filter(city => country.capital.exists((x1) => x1 == city.id)).filter(city => city.population > 8000000).map(city => (country.name, city.name)))

As you can see, the query is written exactly in the same way as we would write it using inmutable data structures and higher-order functions (disguised as for-comprehensions). But this query is embedded in a `quote` block, so what we are actually building is a generic query, which works for the different compilation targets of `quill`: currently, SQL and Cassandra QL. The SQL target actually supports doobie, so that we can execute the `quill` query using a _doobie context_ `dc` as follows:

In [4]:
dc.run(largeCapitals)
    .transact(xa)
    .unsafeRunSync

cmd3.sc:1: SELECT country.name, city.name FROM Country country, City city WHERE country.capital = city.id AND city.population > 8000000
val res3 = dc.run(largeCapitals).transact(xa).unsafeRunSync
                 ^

res3: List[(String, String)] = List(
  ("Indonesia", "Jakarta"),
  ("South Korea", "Seoul"),
  ("Mexico", "Ciudad de M\u00e9xico"),
  ("Russian Federation", "Moscow")
)

The quill-to-doobie compiler shows us the generated SQL query, which, as you can see, is optimal! As we told you, this is thanks to the complex rewriting machinery of the Scala AST, which is performed through macros. Can we get rid of macros, somehow? With the [tagless-final approach](Variation6a.QUEΛ.ipynb), yes, we can!